In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pyarrow
from dataclasses import dataclass
import yaml
from typing import Union

In [2]:
os.getcwd()

'C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01\\notebooks'

In [3]:
os.chdir("..")
os.getcwd()

'C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01'

## Загрузка конфигураций

In [4]:
# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

## Класс `DropBatchProcessor`
- модуль `data_generator.fraud.drops.processor`
- Обработка полученной дропом партии (батча) денег

In [5]:
class DropBatchHandler:
    """
    Обработка полученной дропом партии (батча) денег
    ---------------------------
    drop_type: str. 'distributor' или 'purchaser'
    amt_hand: DropAmountHandler. Генератор сумм входящих/исходящих транзакций, сумм снятий.
              Управление балансом текущего дропа.
    behav_hand: DistBehaviorHandler | PurchBehaviorHandler.
                Управление поведением дропа: распределителя или покупателя.
    create_txn: CreateDropTxn. Создание транзакций.
    declined: bool. По умолчанию False. Отклоняются ли транзакции.
    txns_fm_batch: list. Транзакции дропа в текущем батче.
    """

    def __init__(self, base: DropBaseClasses, create_txn: CreateDropTxn):
        """
        base: DropBaseClasses. Объекты основных классов для дропов.
        create_txn: CreateDropTxn. Создание транзакций.
        """
        self.drop_type = base.drop_type
        self.amt_hand = base.amt_hand
        self.behav_hand = base.behav_hand
        self.create_txn = create_txn
        self.declined = False
        self.txns_fm_batch = []


    def should_decline(self):
        """
        Проверка будет ли отклонена транзакция.
        Возвращает True или False в зависимости от достижения лимитов.
        Также записывает это значение в self.declined
        """
        self.declined = self.create_txn.limit_reached()
        # print("is_declined", self.declined)
        return self.declined


    def reset_cache(self, all=False):
        """
        Сброс кэша.
        --------
        all: bool. Если True то сбрасывает атрибуты: txns_fm_batch, declined.
             Если False то declined не сбрсывает
             Также передается в методы классов:
             DistBehaviorHandler | PurchBehaviorHandler,
             DropAmountHandler.
        """
        behav_hand = self.behav_hand
        amt_hand = self.amt_hand

        self.txns_fm_batch = []
        amt_hand.reset_cache(all=all)
        behav_hand.reset_cache(all=all)

        if not all:
            return
        
        self.declined = False


    def distributor(self):
        """
        Обработка партии(батча) денег полученных дропом
        распределителем.
        """
        behav_hand = self.behav_hand
        amt_hand = self.amt_hand
        create_txn = self.create_txn
        
    
        while amt_hand.balance > 0:
            declined = self.should_decline() # будет ли отклонена транзакция
            behav_hand.guide_scenario()

            if behav_hand.to_crypto: # перевод на криптобиржу или нет
                txn_out = create_txn.purchase(declined=declined)
            else: # Иначе перевод/снятие
                to_drop = behav_hand.to_drop # Пробовать ли перевести другому дропу.
                txn_out = create_txn.trf_or_atm(receive=False, 
                                    to_drop=to_drop, declined=declined)
            # Добавляем в список транз-ций батча   
            self.txns_fm_batch.append(txn_out)

            # Сколько попыток будет после первой откл. транз-ции
            behav_hand.attempts_after_decline()

            # Если это не первая отклоненная транзакция, то вычитаем попытку
            # совершить транзакцию после отклонения
            behav_hand.deduct_attempts()
            
            # Решение об остановке после отклоненной транзакции
            if behav_hand.stop_after_decline():
                break
            

    def purchaser(self):
        """
        Обработка партии(батча) денег полученных дропом
        покупателем.
        """
        behav_hand = self.behav_hand
        amt_hand = self.amt_hand
        create_txn = self.create_txn
        
        while amt_hand.balance > 0:
            declined = self.should_decline() # будет ли отклонена транзакция

            txn_out = create_txn.purchase(declined=declined)
            self.txns_fm_batch.append(txn_out)
            
            # Сколько попыток будет после первой откл. транз-ции
            behav_hand.attempts_after_decline()

            # Если это не первая отклоненная транзакция, то вычитаем попытку
            # совершить транзакцию после отклонения
            behav_hand.deduct_attempts()
            
            # Решение об остановке после отклоненной транзакции
            if behav_hand.stop_after_decline():
                break


    def process_batch(self):
        """
        Вызов соответствующего типу дропа метода для обработки
        батча денег.
        Метод выбирается исходя из self.drop_type.
        ---------
        """
        drop_type = self.drop_type

        if drop_type == "distributor":
            self.distributor()
        elif drop_type == "purchaser":
            self.purchaser()

**Тест `DropBatchProcessor`**

In [2]:
# Временный импорт
import os
import yaml
import pandas as pd
import numpy as np
os.chdir("..")

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [3]:
from data_generator.fraud.drops.build.config import DropConfigBuilder
from data_generator.fraud.drops.build.builder import DropBaseClasses
from data_generator.fraud.drops.txns import CreateDropTxn
from data_generator.fraud.drops.processor import DropBatchHandler
from data_generator.general_time import pd_timestamp_to_unix

drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)
configs = drop_cfg_build.build_dist_cfg()
# configs = drop_cfg_build.build_purch_cfg()

base_agg1 = DropBaseClasses(drop_type="distributor", configs=configs)
# base_agg1 = DropBaseClasses(drop_type="purchaser", configs=configs)
base_agg1.build_all()
# base_agg1.build_all(drop_type="purchaser")
acc_hand1 = base_agg1.acc_hand
amt_hand1 = base_agg1.amt_hand
part_data1 = base_agg1.part_data
time_hand1 = base_agg1.time_hand
behav_hand1 = base_agg1.behav_hand

cr_drop_txn1 = CreateDropTxn(configs=configs, base=base_agg1)
drop_clients = configs.clients

batch_hand = DropBatchHandler(base=base_agg1, create_txn=cr_drop_txn1)

for client in drop_clients.iloc[[8]].itertuples():
    part_data1.client_info = client
    acc_hand1.client_id = client.client_id
    acc_hand1.get_account(own=True)
part_data1.client_info

Pandas(Index=8, client_id=3565, district_id=44, birth_date='1962-07-29', sex='male', region='Ставропольский', area='Ставрополь', timezone='UTC+3', lat=45.0445439, lon=41.9690168, population=398266, home_ip='2.60.13.50', geometry=<MULTIPOLYGON (((41.81 45.006, 41.811 45.006, 41.813 45.007, 41.815 45.007, ...>)

In [4]:
def reset_caches(cr_drop_txn, behav_hand, amt_hand, time_hand, part_data, batch_hand):
    cr_drop_txn.reset_cache(only_counters=False)
    behav_hand.reset_cache(all=True)
    amt_hand.reset_cache(all=True) # batch_txns здесь
    time_hand.reset_cache()
    part_data.reset_cache()
    batch_hand.reset_cache(all=True)

In [4]:
# acc_hand1.outer_accounts.loc[~(acc_hand1.outer_accounts.isin(acc_hand1.used_accounts))].sample(1)
# acc_hand1.outer_accounts.loc[~(acc_hand1.outer_accounts.isin(acc_hand1.used_accounts))]
# acc_hand1.reset_cache()
acc_hand1.used_accounts

Series([], Name: account_id, dtype: object)

**`should_decline`**  
- cr_drop_txn1.out_txns = cr_drop_txn1.out_lim
- cr_drop_txn1.in_txns = cr_drop_txn1.in_lim
- cr_drop_txn1.out_txns = cr_drop_txn1.out_lim - 1
- cr_drop_txn1.in_txns = cr_drop_txn1.in_lim - 1
- cr_drop_txn1.out_txns = cr_drop_txn1.out_lim + 1
- cr_drop_txn1.in_txns = cr_drop_txn1.in_lim + 1

In [5]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1, batch_hand)
# cr_drop_txn1.out_txns = cr_drop_txn1.out_lim
# cr_drop_txn1.in_txns = cr_drop_txn1.in_lim
# cr_drop_txn1.out_txns = cr_drop_txn1.out_lim - 1
# cr_drop_txn1.in_txns = cr_drop_txn1.in_lim - 1
# cr_drop_txn1.out_txns = cr_drop_txn1.out_lim + 1
# cr_drop_txn1.in_txns = cr_drop_txn1.in_lim + 1
batch_hand.should_decline()

True

### **Тест метода `distributor`**

**баланс > trf_max** - выводить кол-во попыток через print внутри atts_after_decline()  
4 кейса
- scen=`"split_transfer"`, in_chunks = `True`
- scen=`"atm+transfer`", in_chunks = `True`
- scen=`"atm"`, in_chunks = `False`
- scen=`"transfer"`, in_chunks = `False`

In [10]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1, batch_hand)
acc_hand1.reset_cache()
all_txns1 = []
start_time = pd.to_datetime("2025-07-02 11:15:00", format="%Y-%m-%d %H:%M:%S")
time_hand1.last_unix = pd_timestamp_to_unix(start_time)
time_hand1.start_unix = pd_timestamp_to_unix(start_time)

In [11]:
while not batch_hand.declined:
    amt_hand1.balance = behav_hand1.trf_max + 1000
    behav_hand1.scen = "transfer"
    behav_hand1.in_chunks = False

    batch_hand.distributor()
    txns_fm_batch1 = batch_hand.txns_fm_batch
    all_txns1.extend(txns_fm_batch1)
    batch_hand.reset_cache(all=False)

attempts 3


In [4]:
all_txns_df = pd.DataFrame(all_txns1)
print(all_txns_df.shape[0])
all_txns_df

**С генерацией входящих транзакций и рандомизацией поведения**
- реализация через цикл наподобие run_drop_lifecycle()

In [7]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1, batch_hand)
acc_hand1.reset_cache()
all_txns1 = []

while True:
    declined = batch_hand.declined
    receive_txn1 = cr_drop_txn1.trf_or_atm(dist=True, receive=True, to_drop=False, declined=declined)
    all_txns1.append(receive_txn1)
    if declined:
        break
        
    behav_hand1.sample_scenario() # выбрать сценарий
    behav_hand1.in_chunks_val() # транзакции по частям или нет 
    
    batch_hand.process_batch()
    txns_fm_batch1 = batch_hand.txns_fm_batch
    all_txns1.extend(txns_fm_batch1)
    batch_hand.reset_cache(all=False)

In [9]:
all_txns_df = pd.DataFrame(all_txns1)
print(all_txns_df.shape[0])
all_txns_df #.query("type != 'inbound'").shape

16


(11, 19)

### **Тест метода `purchaser`**

**баланс > amt_max** - выводить кол-во попыток через print внутри atts_after_decline()
- scen=`"split_money"`, in_chunks = `True`
- scen=`"transfer"`, in_chunks = `False`

In [7]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1, batch_hand)
acc_hand1.reset_cache()
all_txns2 = []
start_time = pd.to_datetime("2025-07-02 11:15:00", format="%Y-%m-%d %H:%M:%S")
time_hand1.last_unix = pd_timestamp_to_unix(start_time)
time_hand1.start_unix = pd_timestamp_to_unix(start_time)

In [8]:
while not batch_hand.declined:
    amt_hand1.balance = behav_hand1.amt_max + 1000
    behav_hand1.scen = "one_purchase"
    behav_hand1.in_chunks = False

    batch_hand.purchaser()
    txns_fm_batch2 = batch_hand.txns_fm_batch
    all_txns2.extend(txns_fm_batch2)
    batch_hand.reset_cache(all=False)

attempts 0


In [10]:
all_txns_df2 = pd.DataFrame(all_txns2)
print(all_txns_df2.shape[0])
all_txns_df2

**`purchaser` с генерацией входящих транзакций и рандомизацией поведения**
- реализация через цикл наподобие run_drop_lifecycle()

In [7]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1, batch_hand)
acc_hand1.reset_cache()
all_txns3 = []

while True:
    declined = batch_hand.declined
    receive_txn3 = cr_drop_txn1.trf_or_atm(receive=True, to_drop=False, declined=declined)
    all_txns3.append(receive_txn3)
    if declined:
        break
        
    behav_hand1.sample_scenario() # выбрать сценарий
    behav_hand1.in_chunks_val() # транзакции по частям или нет 
    
    batch_hand.process_batch()
    txns_fm_batch3 = batch_hand.txns_fm_batch
    all_txns3.extend(txns_fm_batch3)
    batch_hand.reset_cache(all=False)

In [33]:
all_txns_df3 = pd.DataFrame(all_txns3)
print(all_txns_df3.shape[0])
all_txns_df3

# from_7th = all_txns_df3.iloc[7:all_txns_df3.index.max()+1].copy()[["type", "amount", "status"]] #.query("type != 'inbound'").shape
# from_7th["balance"] = -from_7th["amount"]
# from_7th.loc[from_7th.index.min(), "balance"] = from_7th.loc[from_7th.index.min(), "amount"]
# from_7th["balance"] = from_7th["balance"].cumsum()
# from_7th

In [29]:
amt_hand1.balance

np.float64(5700.0)

## Класс `DropLifecycleManager`
- модуль `data_generator.fraud.drops.simulator`
- Полный жизненный цикл дропа

In [ ]:
class DropLifecycleManager:
    """
    Управление полным жизненный циклом одного дропа.
    ------------------
    drop_type: str. 'distributor' или 'purchaser'
    acc_hand: DropAccountHandler. Генератор номеров счетов входящих/исходящих транзакций.
              Учет использованных счетов.
    amt_hand: DropAmountHandler. Генератор сумм входящих/исходящих транзакций, сумм снятий.
              Управление балансом текущего дропа.
    time_hand: DropTimeHandler.
               Управление временем транзакций дропа.
    behav_hand: DistBehaviorHandler | PurchBehaviorHandler.
                Управление поведением дропа: распределителя или покупателя.
    part_data: DropTxnPartData.
               Генерация части данных о транзакции дропа.
    behav_hand: DistBehaviorHandler | PurchBehaviorHandler.
                Управление поведением дропа: распределителя или покупателя.
    create_txn: CreateDropTxn. Создание транзакций.
    batch_hand: DropBatchHandler. Обработка полученной дропом партии (батча) денег
    drop_txns: list. Созданные транзакции дропа.
    """
    def __init__(self, base: DropBaseClasses, create_txn: CreateDropTxn):
        """
        base: DropBaseClasses. Объекты основных классов для дропов.
        create_txn: CreateDropTxn. Создание транзакций.
        """
        self.drop_type = base.drop_type
        self.acc_hand = base.acc_hand
        self.amt_hand = base.amt_hand
        self.time_hand = base.time_hand
        self.part_data = base.part_data
        self.behav_hand = base.behav_hand
        self.create_txn = create_txn
        self.batch_hand = DropBatchHandler(base=base, create_txn=create_txn)
        self.drop_txns = []


    def reset_all_caches(self):
        """
        Сброс кэшей когда активность дропа закончена совсем
        """
        # Сброс всего кэша batch_hand включает в себя полный сброс кэша
        # в behav_hand и amt_hand
        self.batch_hand.reset_cache(all=True)
        self.time_hand.reset_cache()
        self.part_data.reset_cache()
        self.create_txn.reset_cache()
        self.drop_txns = []
        

    def run_drop_lifecycle(self):
        # создать счет дропа, записать is_drop = True в таблице acc_hand.accounts
        acc_hand = self.acc_hand
        # получить номер счета дропа. Пишется в атрибут acc_hand.account
        acc_hand.get_account(own=True) 
        acc_hand.label_drop() # помечаем клиента как дропа в таблице acc_hand.accounts
        
        behav_hand = self.behav_hand
        batch_hand = self.batch_hand
        create_txn = self.create_txn

        while True:
            declined = batch_hand.declined # статус транзакции. будет ли она отклонена
            # входящая транзакция. Новый батч денег.
            receive_txn = create_txn.trf_or_atm(declined=declined, \
                                                to_drop=False, receive=True) 
            drop_txns = self.drop_txns
            drop_txns.append(receive_txn)
            # если у дропа достигнут лимит то транзакции отклоняются. 
            # Если входящая отклонена, дропу больше не пытаются послать деньги
            if declined: 
                break

            behav_hand.sample_scenario() # выбрать сценарий
            behav_hand.in_chunks_val() # транзакции по частям или нет

            batch_hand.process_batch() # обработка полученного батча

            txns_fm_batch = batch_hand.txns_fm_batch
            drop_txns.extend(txns_fm_batch)
            # сброс кэша после завершения обработки батча
            batch_hand.reset_cache(all=False)

## **Тест `DropLifecycleManager`**

In [1]:
# Временный импорт
import os
import yaml
import pandas as pd
import numpy as np
os.chdir("..")

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [10]:
from data_generator.fraud.drops.build.config import DropConfigBuilder
from data_generator.fraud.drops.build.builder import DropBaseClasses
from data_generator.fraud.drops.txns import CreateDropTxn
from data_generator.fraud.drops.simulator import DropLifecycleManager

drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, time_cfg=time_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)
# configs = drop_cfg_build.build_dist_cfg()
configs = drop_cfg_build.build_purch_cfg()

# base_agg1 = DropBaseClasses(drop_type="distributor", configs=configs)
base_agg1 = DropBaseClasses(drop_type="purchaser", configs=configs)
base_agg1.build_all()
acc_hand1 = base_agg1.acc_hand
# amt_hand1 = base_agg1.amt_hand
part_data1 = base_agg1.part_data
# time_hand1 = base_agg1.time_hand
# behav_hand1 = base_agg1.behav_hand

cr_drop_txn1 = CreateDropTxn(configs=configs, base=base_agg1)
drop_clients = configs.clients

life_manager = DropLifecycleManager(base=base_agg1, create_txn=cr_drop_txn1)
batch_hand = life_manager.batch_hand

for client in drop_clients.iloc[[8]].itertuples():
    part_data1.client_info = client
    acc_hand1.client_id = client.client_id
    acc_hand1.get_account(own=True)
part_data1.client_info
print("ready for tests")

ready for tests


In [11]:
# def reset_caches(cr_drop_txn, behav_hand, amt_hand, time_hand, part_data):
#     cr_drop_txn.reset_cache()
#     behav_hand.reset_cache(all=False)
#     amt_hand.reset_cache(life_end=True) # batch_txns здесь
#     time_hand.reset_cache()
#     part_data.reset_cache()

**`run_drop_lifecycle`**  

In [12]:
all_txns = []
life_manager.run_drop_lifecycle()
drop_txns = life_manager.drop_txns
all_txns.extend(drop_txns)
life_manager.reset_all_caches() # сброс всего кэша после завершения активности дропа

In [13]:
all_txns_df = pd.DataFrame(all_txns)
all_txns_df

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,3872,2025-01-06 00:35:00,1736123700,34100.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,13665.0,False,False,approved,not applicable
1,3872,2025-01-06 02:36:00,1736130960,19000.0,purchase,ecom,misc_net,True,6791.0,Уфа,54.734853,55.957865,2.60.14.82,6652.0,NaN,False,False,approved,not applicable
2,3872,2025-01-06 03:16:00,1736133360,5000.0,purchase,ecom,travel_net,True,6825.0,Уфа,54.734853,55.957865,2.60.14.82,6651.0,NaN,False,False,approved,not applicable
3,3872,2025-01-06 05:04:00,1736139840,6000.0,purchase,ecom,misc_net,True,6829.0,Уфа,54.734853,55.957865,2.60.14.82,6652.0,NaN,False,False,approved,not applicable
4,3872,2025-01-06 08:00:00,1736150400,4100.0,purchase,ecom,travel_net,True,6824.0,Уфа,54.734853,55.957865,2.60.14.82,6651.0,NaN,False,False,approved,not applicable
5,3872,2025-01-06 09:46:00,1736156760,33900.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,13665.0,False,False,approved,not applicable
6,3872,2025-01-07 01:17:00,1736212620,33900.0,purchase,ecom,shopping_net,True,6828.0,Уфа,54.734853,55.957865,2.60.14.82,6652.0,NaN,False,False,approved,not applicable
7,3872,2025-01-07 04:11:00,1736223060,6100.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,13665.0,False,False,approved,not applicable
8,3872,2025-01-07 05:14:00,1736226840,6100.0,purchase,ecom,shopping_net,True,6898.0,Уфа,54.734853,55.957865,2.60.14.82,6651.0,NaN,False,False,approved,not applicable
9,3872,2025-01-07 07:53:00,1736236380,16000.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,13665.0,False,False,approved,not applicable


In [5]:
life_manager.drop_txns

[]

In [5]:
behav_hand1.scen, behav_hand1.in_chunks, behav_hand1.attempts

(None, None, 0)

In [6]:
cr_drop_txn1.last_txn, cr_drop_txn1.in_txns, cr_drop_txn1.out_txns

(None, 0, 0)

In [7]:
amt_hand1.batch_txns, amt_hand1.chunk_size, amt_hand1.balance, amt_hand1.last_amt, amt_hand1.first_decl, amt_hand1.declined_txns

(0, 0, 0, 0, 0, 0)

In [8]:
time_hand1.start_unix, time_hand1.last_unix, time_hand1.in_txns, time_hand1.out_txns,

(0, 0, 0, 0)

In [9]:
part_data1.last_txn

In [12]:
# правильный account_id в aсс_hand.account
own_id = acc_hand1.client_id
own_acc_id = acc_hand1.accounts.query("client_id == @own_id")["account_id"].iat[0]
assert acc_hand1.account == own_acc_id, "aсс_hand.account does not belong to client."

In [13]:
# клиент помечен как дроп в accounts
acc_hand1.accounts.query("client_id == @own_id")

,client_id,account_id,is_drop
2057,2174,12057,True


## Класс `DropSimulator`
- модуль `data_generator.fraud.drops.simulator`
- Генерация множества дропов

In [ ]:
# class DropSimulator:
#     """
#     Генерация активности множества дропов.
#     ------------------------
#     drop_clients: pd.DataFrame. Клиенты которые будут дропами.
#     part_data: DropTxnPartData.
#             Генерация части данных о транзакции дропа.
#     acc_hand: DropAccountHandler. Генератор номеров счетов входящих/исходящих транзакций.
#             Учет использованных счетов.
#     life_manager: DropLifecycleManager. Управление полным жизненный циклом одного дропа.
#     """
#     def __init__(self, base_cfg, configs, base, create_txn):
#         """
#         base_cfg: dict. Конфиги из base.yaml
#         configs: DropDistributorCfg | DropPurchaserCfg.
#                  Конфиги и данные для создания дроп транзакций.
#         base: Объекты основных классов для дропов. 
#         create_txn: CreateDropTxn. Создание транзакций.
#         """
#         self.base_cfg = base_cfg
#         self.drop_type = base.drop_type
#         self.drop_clients = configs.clients
#         self.part_data = base.part_data
#         self.acc_hand = base.acc_hand
#         self.life_manager = DropLifecycleManager(base=base, create_txn=create_txn)
#         self.all_txns = []
    
#     def write_to_file(self, data, category, file_key):
#         """
#         Запись данных в файл по пути из yaml конфига.
#         data: pd.DataFrame | gpd.DataFrame. Данные.
#         category: str. Категория файлов в yaml конфиге например:
#                   'cleaned_data', 'generated_data'. Соответсвует
#                   структуре папок в data/
#         file_key: str. Ключ к полному пути конкретного файла в категории.
#         """
#         path = self.base_cfg["data_paths"][category][file_key]
#         file_type = path.split(".")[-1]

#         if file_type == "csv":
#             return data.to_csv(path, index=False)
        
#         if file_type == "gpkg":
#             return data.to_file(path, layer="layer_name", driver="GPKG")
        
#         if file_type == "parquet":
#             return data.to_parquet(path, engine="pyarrow")


#     def run(self):
#         """
#         Полная генерация активности дропов соответсвующего типа
#         """
#         drop_clients = self.drop_clients
#         progress_bar = create_progress_bar(drop_clients)
#         part_data = self.part_data
#         acc_hand = self.acc_hand
#         life_manager = self.life_manager
#         all_txns = self.all_txns

#         # Итерируемся через семплированных клиентов под дроп
#         for client in drop_clients.itertuples():
#             # Запись данных текущего клиента в атрибуты
#             # некоторых классов
#             part_data.client_info = client
#             acc_hand.client_id = client.client_id

#             # Генерация полного цикла активности одного дропа
#             life_manager.run_drop_lifecycle()
#             # Запись транзакций дропа в общий список
#             drop_txns = life_manager.drop_txns
#             all_txns.extend(drop_txns)

#             # Сброс кэша дропа для следующей итерации
#             life_manager.reset_all_caches()
#             progress_bar.update(1)
        
#         # Запись измененного датафрейма accounts в csv файл
#         # Путь указывается в base.yaml
#         accounts = acc_hand.accounts
#         self.write_to_file(data=accounts, category="generated_data", \
#                            file_key="accounts")
        
#         # Запись всех созданных транзакций дропов в parquet файл
#         # Путь указывается в base.yaml
#         all_txns_df = pd.DataFrame(self.all_txns)

#         if self.drop_type == "distributor":
#             self.write_to_file(data=all_txns_df, category="generated_data", \
#                            file_key="dist_drop_txns")
            
#         elif self.drop_type == "purchaser":
#             self.write_to_file(data=all_txns_df, category="generated_data", \
#                            file_key="purch_drop_txns")

**Тест `DropSimulator`**

In [1]:
# Временный импорт
import os
import yaml
import pandas as pd
import numpy as np
import geopandas as gpd
import pyarrow
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

os.chdir("..")

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
with open("./config/legit.yaml") as f:
    legit_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

client_devices = pd.read_csv("./data/base/client_devices.csv")
print("OK")

OK


In [2]:
# datetime_suffix = datetime.now().strftime("%Y-%m-%d_%H%M%S")
# datetime_suffix = "2025-07-20_110919"
# run_dir = "generation_run_" + datetime_suffix
# history_dir = base_cfg["data_paths"]["generated"]["history"]
# run_dir_path = os.path.join(history_dir, run_dir)
# if not os.path.exists(run_dir_path):
#     os.mkdir(run_dir_path)
run_dir = "./data/generated/history/generation_run_2025-07-21_154309"
print("OK")

OK


In [3]:
from data_generator.fraud.drops.build.config import DropConfigBuilder
from data_generator.fraud.drops.build.builder import DropBaseClasses
from data_generator.fraud.drops.txns import CreateDropTxn
from data_generator.fraud.drops.simulator import DropLifecycleManager, DropSimulator
from data_generator.fraud.recorder import FraudTxnsRecorder

drop_type = "distributor"
# drop_type = "purchaser"

drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, legit_cfg=legit_cfg, time_cfg=time_cfg, \
                                   fraud_cfg=fraud_cfg, drop_cfg=drops_cfg, run_dir=run_dir)

if drop_type == "distributor":
    configs = drop_cfg_build.build_dist_cfg()
elif drop_type == "purchaser":
    configs = drop_cfg_build.build_purch_cfg()

base = DropBaseClasses(drop_type=drop_type, configs=configs)
base.build_all()
# acc_hand1 = base.acc_hand
# amt_hand1 = base.amt_hand
# part_data1 = base.part_data
# time_hand1 = base.time_hand
# behav_hand1 = base.behav_hand

create_txn = CreateDropTxn(configs=configs, base=base)
drop_clients = configs.clients
txn_recorder = FraudTxnsRecorder(configs)

# life_manager = DropLifecycleManager(base=base, create_txn=create_txn)
# batch_hand = life_manager.batch_hand
drop_sim = DropSimulator(base_cfg=base_cfg, configs=configs, base=base, create_txn=create_txn, txn_recorder=txn_recorder)
print("ready for tests")

ready for tests


In [4]:
drop_sim.run()
all_txns = txn_recorder.all_txns

Generating distributor drops: 100%|██████████| 2/2 [00:00<00:00, 30.29it/s]


In [5]:
all_txns.dtypes

client_id                 int64
txn_time         datetime64[ns]
unix_time                 int64
amount                  float64
type                     object
channel                  object
category                 object
online                     bool
merchant_id             float64
trans_city               object
trans_lat               float64
trans_lon               float64
trans_ip                 object
device_id               float64
account                 float64
is_fraud                   bool
is_suspicious              bool
status                   object
rule                     object
dtype: object

In [5]:
drop_cfg_build.estimate_drops_count(drop_type)

2

In [6]:
all_txns.shape, all_txns.drop_duplicates().shape

((30, 19), (30, 19))

In [6]:
drop_clients.shape, drop_sim.drop_clients.shape

((2, 11), (2, 11))

In [8]:
# drop_sim.all_txns

In [9]:
all_txns.head(2)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2233,2025-01-14 10:01:00,1736848860,17800.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12116.0,False,False,approved,not applicable
1,2233,2025-01-14 11:26:00,1736853960,10000.0,withdrawal,ATM,not applicable,False,NaN,Пермь,58.010321,56.234178,not applicable,NaN,12116.0,False,False,approved,not applicable


In [23]:
txn_recorder.directory

'./data/generated/history/generation_run_2025-07-21_154309\\purch_drops'

In [10]:
if drop_type == "distributor":
    file_name = "dist_drop_txns.parquet"
elif drop_type == "purchaser":
    file_name = "purch_drop_txns.parquet"

txns_path = os.path.join(txn_recorder.directory, file_name)
# txns_path = txn_recorder.directory
txns_from_run_dir = pd.read_parquet(txns_path)
txns_from_run_dir.head(2)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2233,2025-01-14 10:01:00,1736848860,17800.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12116.0,False,False,approved,not applicable
1,2233,2025-01-14 11:26:00,1736853960,10000.0,withdrawal,ATM,not applicable,False,NaN,Пермь,58.010321,56.234178,not applicable,NaN,12116.0,False,False,approved,not applicable


In [11]:
key_latest = txn_recorder.key_latest
latest_path = base_cfg["data_paths"]["generated"][key_latest]
txns_fm_latest = pd.read_parquet(latest_path)
txns_fm_latest.head(2)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,2233,2025-01-14 10:01:00,1736848860,17800.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,12116.0,False,False,approved,not applicable
1,2233,2025-01-14 11:26:00,1736853960,10000.0,withdrawal,ATM,not applicable,False,NaN,Пермь,58.010321,56.234178,not applicable,NaN,12116.0,False,False,approved,not applicable


In [12]:
all_txns.shape, txns_from_run_dir.shape, txns_fm_latest.shape

((30, 19), (30, 19), (30, 19))

In [10]:
txns_from_run_dir_direct = pd.read_parquet( \
        r"C:\Users\iaros\My_documents\Education\projects\fraud_detection_01\data\generated\history\generation_run_2025-07-21_154309\purch_drops\purch_drop_txns.parquet")

In [11]:
txns_from_run_dir_direct.tail(2)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
33,19,2025-01-17 14:53:00,1737125580,3900.0,purchase,ecom,misc_net,True,6880.0,Барнаул,53.348115,83.779836,2.60.0.19,36.0,NaN,True,False,declined,drop_purchaser
34,19,2025-01-17 15:31:00,1737127860,53800.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,10018.0,True,False,declined,drop_purchaser


In [24]:
txns_from_run_dir_direct02 = pd.read_parquet('./data/generated/history/generation_run_2025-07-21_154309\\purch_drops')
txns_from_run_dir_direct02.tail()

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
32,19,2025-01-17 14:20:00,1.737124e+09,7600.0,purchase,ecom,shopping_net,True,6899.0,Барнаул,53.348115,83.779836,2.60.0.19,36.0,NaN,True,False,declined,drop_purchaser
33,19,2025-01-17 14:53:00,1.737126e+09,3900.0,purchase,ecom,misc_net,True,6880.0,Барнаул,53.348115,83.779836,2.60.0.19,36.0,NaN,True,False,declined,drop_purchaser
34,19,2025-01-17 15:31:00,1.737128e+09,53800.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,10018.0,True,False,declined,drop_purchaser
35,2339,NaT,NaN,NaN,None,None,None,None,NaN,None,NaN,NaN,None,NaN,NaN,None,None,None,None
36,19,NaT,NaN,NaN,None,None,None,None,NaN,None,NaN,NaN,None,NaN,NaN,None,None,None,None


In [8]:
txns_merged = txns_from_run_dir.merge(txns_fm_latest, how="left", on=["client_id", "unix_time"])
txns_merged.loc[txns_merged.trans_city_y.isna()]

,client_id,txn_time_x,unix_time,amount_x,type_x,channel_x,category_x,online_x,merchant_id_x,trans_city_x,...,trans_city_y,trans_lat_y,trans_lon_y,trans_ip_y,device_id_y,account_y,is_fraud_y,is_suspicious_y,status_y,rule_y
35,2339,NaT,NaN,NaN,None,None,None,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,19,NaT,NaN,NaN,None,None,None,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
txns_merged.client_id.unique()

array([1971,  700])

In [13]:
all_txns_minmax = all_txns.client_id.value_counts().agg(["min", "max"])
all_txns_minmax

min    14
max    16
Name: count, dtype: int64

In [14]:
max_client = all_txns.client_id.value_counts().idxmax()
max_client

np.int64(1362)

In [15]:
max_client_txns = all_txns.query("client_id == @max_client")
max_client_txns

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
14,1362,2025-01-16 09:26:00,1737019560,34500.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,11293.0,False,False,approved,not applicable
15,1362,2025-01-16 11:07:00,1737025620,34500.0,withdrawal,ATM,not applicable,False,NaN,Москва,55.753879,37.620373,not applicable,NaN,11293.0,False,False,approved,not applicable
16,1362,2025-01-16 13:09:00,1737032940,26600.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,11293.0,False,False,approved,not applicable
17,1362,2025-01-17 09:52:00,1737107520,15100.0,withdrawal,ATM,not applicable,False,NaN,Москва,55.753879,37.620373,not applicable,NaN,11293.0,False,False,approved,not applicable
18,1362,2025-01-17 12:15:00,1737116100,9000.0,outbound,transfer,not applicable,True,NaN,Москва,55.753879,37.620373,2.60.5.14,2363.0,18751.0,False,False,approved,not applicable
19,1362,2025-01-17 14:23:00,1737123780,2500.0,outbound,transfer,not applicable,True,NaN,Москва,55.753879,37.620373,2.60.5.14,2363.0,23026.0,False,False,approved,not applicable
20,1362,2025-01-17 16:05:00,1737129900,56700.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,11293.0,False,False,approved,not applicable
21,1362,2025-01-17 18:15:00,1737137700,56700.0,withdrawal,ATM,not applicable,False,NaN,Москва,55.753879,37.620373,not applicable,NaN,11293.0,False,False,approved,not applicable
22,1362,2025-01-17 20:53:00,1737147180,12700.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,11293.0,False,False,approved,not applicable
23,1362,2025-01-18 09:00:00,1737190800,6000.0,outbound,transfer,not applicable,True,NaN,Москва,55.753879,37.620373,2.60.5.14,2363.0,16745.0,False,False,approved,not applicable


In [16]:
max_client_txns.type.value_counts()

type
outbound      7
inbound       6
withdrawal    3
Name: count, dtype: int64

In [17]:
all_txns.query("type != 'inbound'")[["client_id","trans_city", "trans_lat", "trans_lon"]].drop_duplicates().shape

(2, 4)

In [18]:
all_txns.client_id.nunique()

2

In [19]:
# проверка что нет дропов у кого не свой девайс

assert all_txns.merge(client_devices, on="device_id").query("client_id_x != client_id_y").empty

In [20]:
# кол-во approved и declined вх. транз. в рамках лимитов

inbound_by_status = all_txns.query("type == 'inbound'").groupby(["client_id","status"], as_index=False).agg({"unix_time":"count"}) \
                                .rename(columns={"unix_time":"txns_count"})
inbound_by_status.head(2)

,client_id,status,txns_count
0,1362,approved,5
1,1362,declined,1


In [21]:
inbound_by_status[["status", "txns_count"]].groupby("status").agg(["min", "max"])

txns_count    
                min max
status                 
approved          4   5
declined          1   1

In [22]:
# кол-во approved и declined исх. транз. в рамках лимитов

outbound_by_status = all_txns.query("type != 'inbound'").groupby(["client_id","status"], as_index=False).agg({"unix_time":"count"}) \
                                .rename(columns={"unix_time":"txns_count"})
outbound_by_status.head(2)

,client_id,status,txns_count
0,1362,approved,8
1,1362,declined,2


In [23]:
outbound_by_status[["status", "txns_count"]].groupby("status").agg(["min", "max"])

txns_count    
                min max
status                 
approved          8   8
declined          1   2

клиенты записаны в дропы. проверка записанного accounts.csv

In [24]:
accounts1 = pd.read_csv("./data/base/accounts.csv")
accounts1.head(2)

,client_id,account_id,is_drop
0,1,10000,False
1,2,10001,False


In [25]:
assert accounts1.loc[accounts1.client_id.isin(drop_clients.client_id)].query("is_drop == False").empty

дропы не пересекаются с другими сегментами: легальные/compr фрод,  дропы другого типа

In [26]:
leg_path = os.path.join(run_dir, "legit", "legit_clients.parquet")
clients_sample = pd.read_parquet(leg_path)
clients_sample.shape

(500, 11)

In [27]:
if drop_type == "distributor":
    other_drops_path = os.path.join(run_dir, "purch_drops", "purchase_drops.parquet") # purchasers
elif drop_type == "purchaser":
    other_drops_path = os.path.join(run_dir, "dist_drops", "dist_drops.parquet") # distributors
other_drops = pd.read_parquet(other_drops_path)
exclude_clients = pd.concat([clients_sample, other_drops], ignore_index=True)

In [28]:
assert exclude_clients.loc[exclude_clients.client_id.isin(all_txns.client_id.unique())].empty

транзакции записаны в файл

In [70]:
drop_txns = pd.read_parquet("./data/generated/latest/dist_drop_txns.parquet", engine="pyarrow") # distributors
# drop_txns = pd.read_parquet("./data/generated/latest/purch_drop_txns.parquet", engine="pyarrow") # purchasers
print(drop_txns.shape)
drop_txns.head(3)

(266, 19)


,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,3861,2025-01-18 02:24:00,1737167040,32100.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,13655.0,False,False,approved,not applicable
1,3861,2025-01-18 04:36:00,1737174960,13000.0,outbound,transfer,not applicable,True,NaN,Улан-Удэ,51.833438,107.584151,2.60.14.72,6634.0,22201.0,False,False,approved,not applicable
2,3861,2025-01-18 05:45:00,1737179100,13000.0,outbound,transfer,not applicable,True,NaN,Улан-Удэ,51.833438,107.584151,2.60.14.72,6633.0,22165.0,False,False,approved,not applicable


In [71]:
drop_txns_hist = pd.read_parquet(r".\data\generated\history\dist_drop_2025-07-20_094115\dist_drop_txns.parquet", engine="pyarrow") # distributors
# drop_txns_hist = pd.read_parquet(r".\data\generated\history\purch_drop_2025-07-20_092817\purch_drop_txns.parquet", engine="pyarrow") # purchasers
print(drop_txns_hist.shape)
drop_txns_hist.head(3)

(266, 19)


,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,3861,2025-01-18 02:24:00,1737167040,32100.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,13655.0,False,False,approved,not applicable
1,3861,2025-01-18 04:36:00,1737174960,13000.0,outbound,transfer,not applicable,True,NaN,Улан-Удэ,51.833438,107.584151,2.60.14.72,6634.0,22201.0,False,False,approved,not applicable
2,3861,2025-01-18 05:45:00,1737179100,13000.0,outbound,transfer,not applicable,True,NaN,Улан-Удэ,51.833438,107.584151,2.60.14.72,6633.0,22165.0,False,False,approved,not applicable


мин и макс разницы во времени у дропов в рамках лимитов

In [29]:
# Добавим колонку со временем предыдущей транзакции каждого клиента
for client in drop_clients.itertuples():
    client_id = client.client_id
    all_txns.loc[all_txns.client_id == client_id, "prev_txn_unix"] \
                = all_txns.loc[all_txns.client_id == client_id, "unix_time"].shift(1)

In [30]:
# all_txns_df.drop(columns="prev_time_diff_m", inplace=True)

In [31]:
# Разница между предыдущей транзакцией клиента в минутах

all_txns["prev_time_diff"] = all_txns.unix_time.sub(all_txns.prev_txn_unix).div(60)
all_txns.head(3)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,...,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule,prev_txn_unix,prev_time_diff
0,2233,2025-01-14 10:01:00,1736848860,17800.0,inbound,transfer,not applicable,True,NaN,not applicable,...,NaN,not applicable,NaN,12116.0,False,False,approved,not applicable,NaN,NaN
1,2233,2025-01-14 11:26:00,1736853960,10000.0,withdrawal,ATM,not applicable,False,NaN,Пермь,...,56.234178,not applicable,NaN,12116.0,False,False,approved,not applicable,1.736849e+09,85.0
2,2233,2025-01-14 13:31:00,1736861460,7000.0,outbound,transfer,not applicable,True,NaN,Пермь,...,56.234178,2.60.8.69,3841.0,16523.0,False,False,approved,not applicable,1.736854e+09,125.0


In [32]:
# timedelta

all_txns["prev_time_diff_m"] = pd.to_timedelta(all_txns["prev_time_diff"], unit="m")
all_txns.head()

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,...,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule,prev_txn_unix,prev_time_diff,prev_time_diff_m
0,2233,2025-01-14 10:01:00,1736848860,17800.0,inbound,transfer,not applicable,True,NaN,not applicable,...,not applicable,NaN,12116.0,False,False,approved,not applicable,NaN,NaN,NaT
1,2233,2025-01-14 11:26:00,1736853960,10000.0,withdrawal,ATM,not applicable,False,NaN,Пермь,...,not applicable,NaN,12116.0,False,False,approved,not applicable,1.736849e+09,85.0,0 days 01:25:00
2,2233,2025-01-14 13:31:00,1736861460,7000.0,outbound,transfer,not applicable,True,NaN,Пермь,...,2.60.8.69,3841.0,16523.0,False,False,approved,not applicable,1.736854e+09,125.0,0 days 02:05:00
3,2233,2025-01-14 14:42:00,1736865720,800.0,outbound,transfer,not applicable,True,NaN,Пермь,...,2.60.8.69,3841.0,15432.0,False,False,approved,not applicable,1.736861e+09,71.0,0 days 01:11:00
4,2233,2025-01-14 16:17:00,1736871420,34300.0,inbound,transfer,not applicable,True,NaN,not applicable,...,not applicable,NaN,12116.0,False,False,approved,not applicable,1.736866e+09,95.0,0 days 01:35:00


In [33]:
all_txns["prev_time_diff_m"].agg(["min","max"])

min   0 days 00:42:00
max   0 days 20:43:00
Name: prev_time_diff_m, dtype: timedelta64[ns]